# Importar Librerias

In [1]:
import pandas as pd
from sqlalchemy import create_engine
import os
import mysql.connector
from dotenv import load_dotenv
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
import numpy as np

# Database connection parameters

In [2]:
conn = mysql.connector.connect(
    host='db-vitrinnea-production.cefwhitrz8zi.us-east-1.rds.amazonaws.com',
    user='admin',
    password='DHrguR7e46xlj',
    database='production_db'
)

# Load data from tables

In [3]:
query_user = """
SELECT id, score FROM users;
"""
users = pd.read_sql(query_user, conn)
print("Users DataFrame:")
print(users.head())

Users DataFrame:
   id  score
0   1    5.0
1  28    5.0
2  29    5.0
3  30    5.0
4  31    5.0


In [4]:
query_orders = """
    SELECT 
        o.id AS order_id, 
        om.user_id, 
        o.order_state_id, 
        om.payment_id,
        o.comment,
        o.total_orden
    FROM 
        orders o
    INNER JOIN 
        order_main om ON o.main_order_id = om.id;
"""
orders = pd.read_sql(query_orders, conn)
print(orders.head())

   order_id  user_id  order_state_id  payment_id comment  total_orden
0        13     28.0               3           6    None          0.0
1        27     54.0               3           6    None          0.0
2        50     95.0               3           1    None          0.0
3        58     96.0               3           8    None          0.0
4        59    108.0               3           8    None          0.0


In [5]:
query_cart = """
SELECT user_id, created_at FROM carts;
"""
carts = pd.read_sql(query_cart, conn)
print("Carts DataFrame:")
print(carts.head())

Carts DataFrame:
   user_id          created_at
0     7120 2022-08-18 12:10:53
1     7120 2022-08-18 12:11:13
2     7120 2022-08-18 12:11:48
3    11447 2022-08-25 18:21:14
4    11394 2022-08-25 21:33:11


# Cargar datos de productos comprados

In [6]:
query_order_items = """
SELECT 
    om.user_id, 
    COUNT(oi.product_id) AS product_count
FROM 
    det_order oi
INNER JOIN 
    orders o ON oi.order_id = o.id
INNER JOIN 
    order_main om ON o.main_order_id = om.id
GROUP BY 
    om.user_id;
"""
order_items = pd.read_sql(query_order_items, conn)
print("Order Items DataFrame:")
print(order_items.head())

conn.close()

Order Items DataFrame:
   user_id  product_count
0     28.0             32
1     54.0              1
2     95.0              1
3     96.0              2
4    108.0              1


# Data preparation

In [7]:
orders['is_canceled'] = orders['order_state_id'].apply(lambda x: 1 if x == 4 else 0)

customer_related_reasons = [
    'CLIENTE_SOLICITO_ANTES_DE_GENERAR_GUIA',
    'CLIENTE_SOLICITO_DESPUES_DE_GENERAR_GUIA_NO_RECOLECTADO',
    'CLIENTE_SOLICITO_DESPUES_DE_GENERAR_GUIA_RECOLECTADO',
    'CLIENTE_NO_ACEPTO_PRODUCTO',
    'IMPOSIBLE_LOCALIZAR_CLIENTE',
    'FRAUDE_DETECTADO_CLIENTE',
    'CLIENTE_SE_COMUNICA_CANCELAR_ENVIO'
]

# Flagging orders with customer-related cancellation reasons

In [8]:
orders['customer_related_cancellation'] = orders.apply(
    lambda row: 1 if row['is_canceled'] == 1 and row['comment'] in customer_related_reasons else 0, 
    axis=1
)

# Payment type flags

In [9]:
orders['paid_by_card'] = orders['payment_id'].apply(lambda x: 1 if x == 6 else 0)
orders['paid_by_cash_and_canceled'] = orders.apply(lambda row: 1 if row['payment_id'] == 8 and row['is_canceled'] == 1 else 0, axis=1)

# Aggregating data

In [10]:
canceled_orders = orders.groupby('user_id')['is_canceled'].sum().reset_index()
customer_related_cancellations = orders.groupby('user_id')['customer_related_cancellation'].sum().reset_index()
order_counts = orders.groupby('user_id')['order_id'].count().reset_index()
total_order_value = orders.groupby('user_id')['total_orden'].sum().reset_index()

order_data = pd.merge(canceled_orders, order_counts, on='user_id')
order_data = pd.merge(order_data, customer_related_cancellations, on='user_id')
order_data = pd.merge(order_data, total_order_value, on='user_id')
order_data = pd.merge(order_data, orders.groupby('user_id')['paid_by_card'].sum().reset_index(), on='user_id')
order_data = pd.merge(order_data, orders.groupby('user_id')['paid_by_cash_and_canceled'].sum().reset_index(), on='user_id')

order_data['cancel_rate'] = order_data['is_canceled'] / order_data['order_id']

# Merge with user data

In [11]:
data = pd.merge(users, order_data, left_on='id', right_on='user_id', how='left')
data = pd.merge(data, carts.groupby('user_id').size().reset_index(name='cart_count'), left_on='id', right_on='user_id', how='left')
data = pd.merge(data, order_items, left_on='id', right_on='user_id', how='left')
print(data.head())

   id  score  user_id_x  is_canceled  order_id  customer_related_cancellation  \
0   1    5.0        1.0        508.0     646.0                            2.0   
1  28    5.0       28.0          0.0      27.0                            0.0   
2  29    5.0       29.0          0.0       1.0                            0.0   
3  30    5.0        NaN          NaN       NaN                            NaN   
4  31    5.0        NaN          NaN       NaN                            NaN   

   total_orden  paid_by_card  paid_by_cash_and_canceled  cancel_rate  \
0     11442.23         124.0                      356.0     0.786378   
1       218.38           5.0                        0.0     0.000000   
2         0.00           1.0                        0.0     0.000000   
3          NaN           NaN                        NaN          NaN   
4          NaN           NaN                        NaN          NaN   

   user_id_y  cart_count  user_id  product_count  
0        1.0         4.0     

# Fill missing values for users without orders or carts

In [12]:
data['cancel_rate'].fillna(0, inplace=True)
data['cart_count'].fillna(0, inplace=True)
data['customer_related_cancellation'].fillna(0, inplace=True)
data['total_orden'].fillna(0, inplace=True)
data['paid_by_card'].fillna(0, inplace=True)
data['paid_by_cash_and_canceled'].fillna(0, inplace=True)
data['product_count'].fillna(0, inplace=True)

# Define features and target

In [13]:
features = ['cancel_rate', 'cart_count', 'customer_related_cancellation', 'total_orden', 'paid_by_card', 'paid_by_cash_and_canceled', 'product_count']
target = 'score'

# Prepare data for training

In [14]:
X = data[features]
y = data[target]

# Normalize the features

In [15]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

# Split data into training and testing sets

In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Build and train Gradient Boosting Machines model

In [17]:
gbm = GradientBoostingRegressor(n_estimators=100, learning_rate=0.1, max_depth=3, random_state=42)
gbm.fit(X_train, y_train)

GradientBoostingRegressor(random_state=42)

# Predict and evaluate

In [18]:
predictions = gbm.predict(X_test)
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error: {mae}')

Mean Absolute Error: 0.0180222709767356


# Predict and update user scores

In [19]:
data['predicted_score'] = gbm.predict(X)

# Guardar score y user_id en un archivo CSV

In [20]:
csv_filename = 'user_scores_with_features.csv'
data.to_csv(csv_filename, index=False)
print(f"Data saved to {csv_filename}")

Data saved to user_scores_with_features.csv
